This is an example of one of my best bots. Credit to [Lindada](https://www.kaggle.com/a763337092/pull-vegas-slot-machines-add-weaken-rate-continue5) for his great starter workbook!

For an explanation of the logic behind my code, see the accompanying [discussion post](https://www.kaggle.com/c/santa-2020/discussion/216748).

In [ ]:
!pip install --upgrade pip

In [ ]:
!pip install kaggle-environments --upgrade

In [ ]:
from kaggle_environments import make
env = make("mab", debug=True)

In [ ]:
def print_rounds(file1, file2, N=5):
    env = make("mab", debug=True)

    p1=0
    p2=0
    draw=0
    for i in range(N):
        env.run([file1, file2])
        p1_score = env.steps[-1][0]['reward']
        p2_score = env.steps[-1][1]['reward']
        p1+=p1_score>p2_score
        p2+=p2_score>p1_score
        draw+=p1_score==p2_score
        env.reset()
        print(f"Round {i+1}: {p1_score} - {p2_score}, {file1} {p1} - {p2} {file2}")
    print(file1 + " : " + str(p1))
    print(file2 + " : " + str(p2))
    print("draws" + " : " + str(draw))

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%writefile example.py

import numpy as np
import pandas as pd
import random, os, datetime, math
from collections import defaultdict

total_reward = 0
bandit_dict = {}

bandit_pulls=[]

for x in range(100):
    bandit_pulls.append(0)

def set_seed(my_seed=27):
    os.environ['PYTHONHASHSEED'] = str(my_seed)
    random.seed(my_seed)
    np.random.seed(my_seed)

def get_next_bandit(T):
    best_bandit = 0
    best_bandit_expected = 0

    for bnd in bandit_dict: 
        gg=(bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss']==1) and (bandit_dict[bnd]['opp']==1)                                                          
        expect = (1.0*bandit_dict[bnd]['win']\
                  -.5+0.5*gg -.00*bandit_dict[bnd]['loss']\
                  + (1.01-0.5*(T/2000)**4)*bandit_dict[bnd]['op_continue'] \
                  + (0.99-0.5*(T/2000)**4)*(bandit_dict[bnd]['opp']-bandit_dict[bnd]['op_continue']-bandit_dict[bnd]['op_switch']-bandit_dict[bnd]['op_bad_switch']-gg)\
                  + (.65-0.1*(T/2000)**3)*bandit_dict[bnd]['op_switch']\
                  + .5*bandit_dict[bnd]['op_bad_switch']) \
                 / (bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp']) \
                * math.pow(0.97, (bandit_dict[bnd]['win'] + bandit_dict[bnd]['loss'] + bandit_dict[bnd]['opp'])/(1+T/2000)**1.75)
        if expect > best_bandit_expected:
            best_bandit_expected = expect
            best_bandit = bnd
    return best_bandit


my_action_list = []
op_action_list = []

op_continue_cnt_dict = defaultdict(int)

def multi_armed_probabilities(observation, configuration):
    global total_reward, bandit_dict,bandit_pulls

    my_pull = random.randrange(configuration['banditCount'])
    if 0 == observation['step']:
        set_seed()
        total_reward = 0
        bandit_dict = {}
        for i in range(configuration['banditCount']):
            bandit_dict[i] = {'win': 1, 'loss': 0, 'opp': 0, 'op_continue': 0, 'op_switch':0, 'op_bad_switch':0}
        
    
    else:
        last_reward = observation['reward'] - total_reward
        total_reward = observation['reward']
        
        my_idx = observation['agentIndex']
        my_last_action = observation['lastActions'][my_idx]
        op_last_action = observation['lastActions'][1-my_idx]
        
        my_action_list.append(my_last_action)
        op_action_list.append(op_last_action)
        
        bandit_pulls[my_last_action]+=1
        bandit_pulls[op_last_action]+=1
        
        if 0 < last_reward:
            bandit_dict[my_last_action]['win'] = bandit_dict[my_last_action]['win'] +1
            
        else:
            bandit_dict[my_last_action]['loss'] = bandit_dict[my_last_action]['loss'] +1
            
        bandit_dict[op_last_action]['opp'] = bandit_dict[op_last_action]['opp'] +1
        
        if observation['step'] >= 3:
            if op_action_list[-1] == op_action_list[-2]:
                bandit_dict[op_last_action]['op_continue'] += 1
            else:
               if bandit_pulls[op_action_list[-1]]>1: 
                bandit_dict[op_action_list[-2]]['op_switch'] += 1
               else: 
                bandit_dict[op_action_list[-2]]['op_bad_switch']+=1
                
        if observation['step']>1600 and last_reward==1:
            return my_last_action
        
    return get_next_bandit(observation['step'])

In [ ]:
print_rounds("example.py","example.py",1)